# Importando Objetos

In [0]:
%run 
./04.objects

# Bibliotecas

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from kafka import KafkaConsumer
import datetime
import pickle

# Consumer

In [0]:
consumer = KafkaConsumer("clean-data", bootstrap_servers=["localhost:9092"])

for message in consumer:
    now = datetime.datetime.now()
    now_min = now.minute
    print(f"Processo iniciado em {now.strftime('%Y-%m-%d %H:%M:%S')}")
    
    value = message.value
    articles_list = pickle.loads(value)

    # Instanciando Pipeline
    pipeline_data = Data_Pipeline_API()

    criteria = Search_Criteria("genomics", ["DNA", "genetic", "treatment"])
    
    if len(articles_list) != 0:
        # Carregando dados da API
        pipeline_data.load_api(articles_list)

        # Salvando em Parquet
        pipeline_data.save_data("raw_data")

        print(f"raw_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")

    # Importando DataFrame
    pipeline_data.load_parquet("raw_data")

    # Limpeza de dados
    pipeline_data.data_cleaning()

    # Salvando Parquet
    pipeline_data.save_data("cleaned_data")
    print(f"cleaned_data atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")

    pipeline_data.delta_table()
    print(f"delta_table atualizado em {now.strftime('%Y-%m-%d %H:%M:%S')}")

    # Disponibilizando no DB
    pipeline_data.to_gold(criteria.keywords)
    print(f"tabelas disponibilizadas em {now.strftime('%Y-%m-%d %H:%M:%S')}")

Processo iniciado em 2024-10-25 23:00:05
raw_data atualizado em 2024-10-25 23:00:05
cleaned_data atualizado em 2024-10-25 23:00:05
delta_table atualizado em 2024-10-25 23:00:05
tabelas disponibilizadas em 2024-10-25 23:00:05


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can